In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=15,
    ).choices[0].message.content    

In [ ]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL'),
)
model_name = os.getenv("MODEL_NAME")

In [ ]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "What is the capital of Germany?"},
    ],
    temperature=0.0,
    top_p=0.8,
    max_tokens=15,
)

In [ ]:
response.choices[0].message.content